In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("data/bronze/crypto_trades")
print("rows:", df.count())
df.orderBy("offset", ascending=False).show(10, truncate=False)

rows: 37
+-------------+---------+------+-----------------------+-------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic        |partition|offset|ts_kafka               |ts_type|key     |value_raw                                                                                                                                                          |
+-------------+---------+------+-----------------------+-------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|crypto.trades|0        |36    |2025-09-02 12:51:13.375|0      |XBT/USDT|{"exchange":"kraken","symbol":"XBT/USDT","price":110749.2,"size":2.19e-05,"side":"sell","order_type":"limit","ts_event":1756842673295,"ts_ingest":1756842673375}   |
|crypto.trades|0        |35    |2025-09